## ⚠ Remarque: ⚠ 
C'est juste un notebook draft pour tester split en train et test & l'évaluation sur BLEU score.

--- > Il faudrait split le train et le test comme fait ici, save le test set de coté. Entrainer le model avec le training set. Puis évaluer sur le test set. 

Là je teste sur un model déjà save qui s'est entrainé sur tout le set de départ (train + test) donc c'est à ne pas prendre en compte !

In [ ]:
# !pip install simplet5

In [49]:
import numpy as np
import pandas as pd
import joblib

from simplet5 import SimpleT5
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import sentence_bleu

## I) Load Dataset


In [2]:
path = "simpleT5_last.tsv"
df = pd.read_csv(path, sep='\t')
df.drop(labels=0, axis=0, inplace=True)

In [3]:
# Rename Columns
df.rename(columns={"Reponse":"target_text", "Question":"source_text"}, inplace=True)

# Add prefix
df['source_text'] = "answer_question: " + df['source_text']

In [4]:
df.head(5)

,source_text,target_text
1,answer_question: Quels sont tes centres d'inte...,Je suis interessé par beaucoup de choses. Nous...
2,answer_question: Quels sont tes centres d'inte...,Je suis interessé par une grande variete de su...
3,answer_question: Quels sont tes sujets preferes,"Mes sujets preferes incluent la robotique, l'i..."
4,answer_question: Quel est ton numero,Je n'ai pas de numero.
5,answer_question: Quel est ton numero,"5446, c'est mon numero maintenant."


## II) Split into train and test set

L'idée: on split comme on a l'habitude de faire, puis on retire du training set les questions qui figurent dans le test set pour les rajouter à ce dernier.

In [5]:
train_df, test_df = train_test_split(df, shuffle=True, test_size=0.02, random_state=21)
train_df.shape, test_df.shape

((35435, 2), (724, 2))

We remove the questions from the training set that we already have in the test set.

In [6]:
# Get names of indexes for which the question is the same in the training set as in the test set
indexNames = train_df[ train_df['source_text'].isin(list(test_df['source_text']) )].index

rows_to_remove = train_df.loc[indexNames]
rows_to_remove.shape

(373, 2)

In [7]:
# Delete these row indexes from dataFrame
train_df.drop(indexNames , inplace=True)
# The size of the training set goes down a bit
train_df.shape, test_df.shape

((35062, 2), (724, 2))

In [8]:
# On peut concatener les Q/A retirées du train au test set
test_df_new = pd.concat([test_df, rows_to_remove], axis=0)
test_df_new.shape

(1097, 2)

In [9]:
test_df_new.head(5)

,source_text,target_text
22732,"answer_question: C'est bon, la période d'adapt...","Oui, j'ai été dans cette situation. Reste posi..."
36113,"answer_question: Je ne suis pas sûre, c'était ...","Cool, je les aime bien, mais elles sont follem..."
17072,"answer_question: Une fois, quand j'étais au ja...",Oh non ! Tu as eu des ennuis ?
29487,answer_question: Les dents de sagesse sont ret...,"Ouille, gargarise-toi avec du whisky"
33595,answer_question: Nous avons dû quitter la mais...,L'ouragan Matthew est aussi passé par là où je...


In [43]:
questions_test = list(test_df_new['source_text'])
answers_test = list(test_df_new['target_text'])

len(answers_test), len(questions_test) 

(1097, 1097)

## III) Load model and test with BLEU score

In [37]:
# Load le model
model = SimpleT5()
modelPATH = "drive/MyDrive/TER/ml/models/SimpleT5.joblib"

model = joblib.load(modelPATH)

In [56]:
# Compute score
score = 0
for i, qst in tqdm(enumerate(questions_test)):
  answer_chatbot = model.predict(qst, num_return_sequences=10, num_beams=10)
  answer_chatbot_splitted = [answer.split() for answer in answer_chatbot]
  real_answer_splitted = answers_test[i].split()
  score += sentence_bleu(answer_chatbot_splitted, real_answer_splitted)
score/= len(questions_test)
print(f'Le score est de {score}')

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Le score est de 0.6297767901291315


In [57]:
print(f'Le score est de {score}')

Le score est de 0.6297767901291315


In [ ]:
# Je laisse ça là au cas ou on en aurait besoin 

# grouped_by_questions = result.groupby('source_text')['target_text'].apply(list)
# questions_test = list(grouped_by_questions.index)
# answers_test = list(grouped_by_questions)
# len(answers_test), len(questions_test) 